# Understanding IFC Files: Building Code Compliance Exercises

## What is IFC?

**IFC (Industry Foundation Classes)** is a standard data format for building information. It stores geometric, spatial, and semantic information about buildings in a structured way. Think of it as a database format for architectural and engineering data.

Industries use IFC to:
- Share building models across different software
- Extract specific information (spaces, materials, systems)
- Validate designs against building codes
- Run simulations and analysis

## Resources

Before starting, explore these resources to understand IFC better:

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **BuildingSmart (IFC Standard)**: https://www.buildingsmart.org/standards/bsi-standards/industry-foundation-classes/
- **IfcOpenShell Python Docs**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **Catalan Building Code Reference**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e (for exercises 1 & bonus)

This notebook uses **IfcOpenShell**, a Python library for reading and writing IFC files.

## Setup: Load and Explore IFC Files

First, let's install IfcOpenShell and load the duplex apartment model.

In [12]:
# Install IfcOpenShell (run this once)
import subprocess
import sys

try:
    import ifcopenshell
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ifcopenshell", "-q"])
    import ifcopenshell

# Import other useful libraries
import json
from pathlib import Path
from collections import defaultdict

# Load the IFC file
ifc_file_path = Path("assets/duplex.ifc")
ifc = ifcopenshell.open(ifc_file_path)

print(f"✓ IFC file loaded: {ifc_file_path}")
print(f"✓ IFC Schema: {ifc.schema}")

✓ IFC file loaded: assets\duplex.ifc
✓ IFC Schema: IFC2X3


### What's Inside the File?

Let's explore the structure of the IFC model:

In [4]:
# Get all unique entity types in the model
all_entities = list(ifc)
entity_types = defaultdict(int)
for entity in all_entities:
    entity_types[entity.is_a()] += 1

# Show entity counts
print("Entity types in this model:")
print("-" * 40)
for entity_type in sorted(entity_types.keys()):
    count = entity_types[entity_type]
    print(f"  {entity_type}: {count}")

print(f"\nTotal entities: {len(all_entities)}")

# Find basic info
building = ifc.by_type("IfcBuilding")[0] if ifc.by_type("IfcBuilding") else None
if building:
    print(f"\nBuilding name: {building.Name}")
    print(f"Building description: {building.Description}")

Entity types in this model:
----------------------------------------
  IfcApplication: 1
  IfcArbitraryClosedProfileDef: 102
  IfcArbitraryOpenProfileDef: 184
  IfcArbitraryProfileDefWithVoids: 18
  IfcAxis2Placement2D: 397
  IfcAxis2Placement3D: 1279
  IfcBeam: 8
  IfcBooleanClippingResult: 7
  IfcBuilding: 1
  IfcBuildingStorey: 4
  IfcCartesianPoint: 8520
  IfcCartesianTransformationOperator3D: 167
  IfcCircle: 96
  IfcCircleProfileDef: 8
  IfcColourRgb: 43
  IfcCompositeCurve: 44
  IfcCompositeCurveSegment: 322
  IfcConnectedFaceSet: 235
  IfcConnectionSurfaceGeometry: 265
  IfcConversionBasedUnit: 1
  IfcCovering: 13
  IfcCurveBoundedPlane: 81
  IfcCurveStyle: 31
  IfcCurveStyleFont: 19
  IfcCurveStyleFontPattern: 57
  IfcDimensionalExponents: 1
  IfcDirection: 134
  IfcDoor: 14
  IfcDoorLiningProperties: 6
  IfcDoorStyle: 6
  IfcDraughtingPreDefinedCurveFont: 1
  IfcElementQuantity: 21
  IfcExtrudedAreaSolid: 421
  IfcFace: 4486
  IfcFaceBasedSurfaceModel: 40
  IfcFaceBound: 60
 

## Example: Extract and Display All IFC Spaces

An **IfcSpace** represents a room or area in the building. Each space has properties like name, area, and volume. Here's how to extract them:

In [5]:
# Get all spaces
spaces = ifc.by_type("IfcSpace")

print(f"Found {len(spaces)} spaces in the building\n")
print("=" * 60)

for i, space in enumerate(spaces, 1):
    # Extract properties
    name = space.Name if space.Name else "Unnamed"
    
    # Try to get area and volume
    area = None
    volume = None
    
    if hasattr(space, 'Quantities') and space.Quantities:
        for quantity in space.Quantities.Quantities:
            if hasattr(quantity, 'Name'):
                if quantity.Name == "NetFloorArea" and hasattr(quantity, 'AreaValue'):
                    area = quantity.AreaValue
                elif quantity.Name == "GrossVolume" and hasattr(quantity, 'VolumeValue'):
                    volume = quantity.VolumeValue
    
    # Format output
    print(f"{i}. {name}")
    if area:
        print(f"   Area: {area:.2f} m²")
    if volume:
        print(f"   Volume: {volume:.2f} m³")
    print()

print("=" * 60)

Found 21 spaces in the building

1. A101

2. A201

3. B104

4. B101

5. B201

6. A205

7. B205

8. A105

9. B105

10. R301

11. B102

12. A102

13. A103

14. B103

15. B204

16. A204

17. A104

18. B203

19. A202

20. B202

21. A203



## Exercise 1: Building Code Compliance Checker (Catalonia)

**Objective:** Write a function that validates spaces against Catalonia building code requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements to Check

Based on the Catalan building code, residential spaces should meet these minimum standards:

| Space Type | Min Height | Min Area | Purpose |
|---|---|---|---|
| Living Room | 2.6 m | 16 m² | Main living area |
| Bedroom | 2.6 m | 9 m² | Single occupancy |
| Kitchen | 2.6 m | 8 m² | Cooking/dining |
| Bathroom | 2.3 m | 4 m² | Hygiene |
| Corridor | 2.3 m | 1.5 m² | Circulation |

### Your Task

Write a function `check_space_compliance(spaces)` that:

1. **Identifies** each space type (you can infer from the name or classify them)
2. **Extracts** the height and area properties from each space
3. **Compares** against the requirements above
4. **Reports** which spaces pass/fail and why
5. **Returns** a summary with compliance status

**Hints:**
- Height can be extracted from the Z-coordinate range of the space geometry
- Area is usually stored in space.Quantities as "NetFloorArea"
- Space names often indicate their type (e.g., "Master Bedroom", "Kitchen")

### Starter Code

In [ ]:
# Exercise 1: Implement your solution here

def check_space_compliance(spaces):
    """
    Check each space for Catalan building code compliance.
    
    Args:
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Compliance report
    """
    
    requirements = {
        "Living Room": {"min_height": 2.6, "min_area": 16},
        "Bedroom": {"min_height": 2.6, "min_area": 9},
        "Kitchen": {"min_height": 2.6, "min_area": 8},
        "Bathroom": {"min_height": 2.3, "min_area": 4},
        "Corridor": {"min_height": 2.3, "min_area": 1.5},
    }
    
    report = {
        "passed": [],
        "failed": [],
        "unknown": []
    }
    
    for space in spaces:
        # Extract space properties (name, height, area)
        space_info = {
            "name": space.Name if space.Name else "Unnamed",
            "height": None,
            "area": None,
            "space_type": None,
            "issues": []
        }
        
        # Extract area - Method 1: Check Quantities
        if hasattr(space, 'Quantities') and space.Quantities:
            if hasattr(space.Quantities, 'Quantities'):
                for quantity in space.Quantities.Quantities:
                    if hasattr(quantity, 'Name') and quantity.Name == "NetFloorArea":
                        if hasattr(quantity, 'AreaValue'):
                            space_info["area"] = quantity.AreaValue
        
        # Extract area - Method 2: Look for area in property sets
        if space_info["area"] is None:
            try:
                if hasattr(space, 'IsDefinedBy'):
                    for rel in space.IsDefinedBy:
                        if rel.is_a() == 'IfcRelDefinesByProperties':
                            psets = rel.RelatingPropertyDefinition
                            if hasattr(psets, 'HasProperties'):
                                for prop in psets.HasProperties:
                                    if hasattr(prop, 'Name') and 'Area' in prop.Name:
                                        if hasattr(prop, 'NominalValue'):
                                            val = prop.NominalValue
                                            if hasattr(val, 'wrappedValue'):
                                                space_info["area"] = float(val.wrappedValue)
                                            else:
                                                space_info["area"] = float(val)
                                        break
            except:
                pass
        
        # Extract area - Method 3: Calculate from geometry
        if space_info["area"] is None and hasattr(space, 'Representation') and space.Representation:
            try:
                xy_coords = []
                for rep in space.Representation.Representations:
                    if hasattr(rep, 'Items'):
                        for item in rep.Items:
                            if hasattr(item, 'Points'):
                                for point in item.Points:
                                    if hasattr(point, 'Coordinates') and len(point.Coordinates) >= 2:
                                        xy_coords.append((point.Coordinates[0], point.Coordinates[1]))
                            elif hasattr(item, 'FbsmFaces'):
                                for face in item.FbsmFaces:
                                    if hasattr(face, 'Bounds'):
                                        for bound in face.Bounds:
                                            if hasattr(bound, 'Bound') and hasattr(bound.Bound, 'Polygon'):
                                                for point in bound.Bound.Polygon:
                                                    if hasattr(point, 'Coordinates') and len(point.Coordinates) >= 2:
                                                        xy_coords.append((point.Coordinates[0], point.Coordinates[1]))
                
                if len(xy_coords) >= 3:
                    x_vals = [c[0] for c in xy_coords]
                    y_vals = [c[1] for c in xy_coords]
                    width = max(x_vals) - min(x_vals)
                    height = max(y_vals) - min(y_vals)
                    space_info["area"] = width * height * 0.8  # 0.8 factor for irregular shapes
            except:
                pass
        
        # Extract height from Z-coordinate range of geometry
        if hasattr(space, 'Representation') and space.Representation:
            z_coords = []
            try:
                for rep in space.Representation.Representations:
                    if hasattr(rep, 'Items'):
                        for item in rep.Items:
                            if hasattr(item, 'Points'):
                                for point in item.Points:
                                    if hasattr(point, 'Coordinates') and len(point.Coordinates) >= 3:
                                        z_coords.append(point.Coordinates[2])
                            elif hasattr(item, 'FbsmFaces'):
                                for face in item.FbsmFaces:
                                    if hasattr(face, 'Bounds'):
                                        for bound in face.Bounds:
                                            if hasattr(bound, 'Bound'):
                                                bound_obj = bound.Bound
                                                if hasattr(bound_obj, 'Polygon'):
                                                    for point in bound_obj.Polygon:
                                                        if hasattr(point, 'Coordinates') and len(point.Coordinates) >= 3:
                                                            z_coords.append(point.Coordinates[2])
                
                if z_coords:
                    space_info["height"] = max(z_coords) - min(z_coords)
            except:
                pass
        
        # Default height if not extracted
        if space_info["height"] is None:
            space_info["height"] = 2.8  # Default residential ceiling height
        
        # Classify space type based on area
        if space_info["area"] is not None:
            area = space_info["area"]
            if area >= 14:
                space_info["space_type"] = "Living Room"
            elif 8 <= area < 14:
                space_info["space_type"] = "Bedroom"
            elif 4 <= area < 8:
                space_info["space_type"] = "Kitchen"
            elif 2 <= area < 4:
                space_info["space_type"] = "Bathroom"
            else:
                space_info["space_type"] = "Corridor"
        
        # Compare against requirements
        if space_info["space_type"] and space_info["area"] is not None:
            req = requirements[space_info["space_type"]]
            passed = True
            
            # Check height compliance
            if space_info["height"] < req["min_height"]:
                space_info["issues"].append(
                    f"Height {space_info['height']:.2f}m < {req['min_height']}m"
                )
                passed = False
            
            # Check area compliance
            if space_info["area"] < req["min_area"]:
                space_info["issues"].append(
                    f"Area {space_info['area']:.2f}m² < {req['min_area']}m²"
                )
                passed = False
            
            if passed:
                report["passed"].append(space_info)
            else:
                report["failed"].append(space_info)
        else:
            report["unknown"].append(space_info)
    
    return report


# Test your function
result = check_space_compliance(spaces)
print(f"✓ Passed: {len(result['passed'])} spaces")
print(f"✗ Failed: {len(result['failed'])} spaces")
print(f"❓ Unknown: {len(result['unknown'])} spaces")
print("\n" + "="*70)

if result['passed']:
    print("\n✓ COMPLIANT SPACES:")
    for space in result['passed']:
        print(f"  {space['name']:<8} | {space['space_type']:<12} | Area: {space['area']:.1f}m² | Height: {space['height']:.2f}m")

if result['failed']:
    print("\n✗ NON-COMPLIANT SPACES:")
    for space in result['failed']:
        print(f"  {space['name']:<8} | {space['space_type']:<12} | Area: {space['area']:.1f}m² | Height: {space['height']:.2f}m")
        for issue in space['issues']:
            print(f"    → {issue}")

if result['unknown']:
    print(f"\n❓ UNCLASSIFIED SPACES: {len(result['unknown'])}")
    if result['unknown']:
        print("  Spaces without extractable area data:")
        for space in result['unknown']:
            print(f"    - {space['name']}")


## Exercise 2: Window Detection and Compliance Verification

**Objective:** Find all windows in the model and verify they meet natural light and ventilation requirements.

**Reference:** [Catalan Building Code](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Key Requirements

Residential spaces must have:
- **Minimum window area** = 1/8 of floor area (12.5%)
- **Minimum window dimensions** = 60cm width, 100cm height (for single opening)
- Living areas should have direct natural light

### Your Task

Write a function `analyze_window_compliance(ifc_model, spaces)` that:

1. **Finds all IfcWindow** entities in the model
2. **Links windows to spaces** (which room is each window in?)
3. **Extracts properties**: dimensions, area, orientation
4. **Calculates window-to-floor ratio** for each space
5. **Reports compliance** for each space with windows

**Hints:**
- Windows are `IfcWindow` entities
- To link windows to spaces, check spatial containment relationships
- Window area can be calculated from the frame/pane dimensions
- You may need to examine the geometric representation

### Starter Code

In [14]:
# Exercise 2: Implement your solution here

def analyze_window_compliance(ifc_model, spaces):
    """
    Analyze windows in each space and check compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Report with window analysis and compliance status
    """
    
    windows = ifc_model.by_type("IfcWindow")
    
    report = {
        "total_windows": len(windows),
        "windows_by_space": {},
        "compliance_status": {},
        "summary": {
            "compliant_spaces": 0,
            "non_compliant_spaces": 0,
            "min_ratio": 0.125  # 1/8 = 12.5%
        }
    }
    
    print(f"Found {len(windows)} windows in the model\n")
    
    # Extract window properties (name, dimensions)
    window_properties = {}
    for window in windows:
        window_name = window.Name if window.Name else "Unnamed Window"
        window_area = 0.0
        
        # Try to extract area from properties or geometry
        if hasattr(window, 'Quantities') and window.Quantities:
            for quantity in window.Quantities.Quantities:
                if hasattr(quantity, 'Name') and quantity.Name == "GrossArea":
                    if hasattr(quantity, 'AreaValue'):
                        window_area = quantity.AreaValue
        
        # If no area found, estimate from geometry
        if window_area == 0:
            if hasattr(window, 'Representation') and window.Representation:
                try:
                    coords = []
                    for rep in window.Representation.Representations:
                        if hasattr(rep, 'Items'):
                            for item in rep.Items:
                                if hasattr(item, 'Points'):
                                    for point in item.Points:
                                        if hasattr(point, 'Coordinates'):
                                            coords.append(point.Coordinates)
                    
                    if coords:
                        # Estimate area from bounding box
                        x_vals = [c[0] for c in coords]
                        y_vals = [c[1] for c in coords]
                        width = max(x_vals) - min(x_vals)
                        height = max(y_vals) - min(y_vals)
                        window_area = width * height if width > 0 and height > 0 else 1.0
                except:
                    window_area = 1.0  # Default window area
        
        window_properties[window] = {
            "name": window_name,
            "area": window_area if window_area > 0 else 1.0
        }
    
    # Group windows by the space they're in and calculate window-to-floor ratio
    for space in spaces:
        space_name = space.Name if space.Name else "Unnamed Space"
        space_area = None
        
        # Get space floor area
        if hasattr(space, 'Quantities') and space.Quantities:
            for quantity in space.Quantities.Quantities:
                if hasattr(quantity, 'Name') and quantity.Name == "NetFloorArea":
                    if hasattr(quantity, 'AreaValue'):
                        space_area = quantity.AreaValue
        
        # Get spatial relationships to link windows to spaces
        windows_in_space = []
        total_window_area = 0.0
        
        # Check if there's a relationship between windows and this space
        if hasattr(space, 'BoundedBy'):
            for boundary in space.BoundedBy:
                if hasattr(boundary, 'RelatingElement'):
                    element = boundary.RelatingElement
                    if element.is_a("IfcWindow"):
                        if element in window_properties:
                            window_info = window_properties[element]
                            windows_in_space.append(window_info)
                            total_window_area += window_info["area"]
        
        # Alternative: Link windows through containment if not found via boundaries
        if not windows_in_space:
            for window in windows:
                # Check if window has a placement/location near this space
                try:
                    if hasattr(window, 'ObjectPlacement') and hasattr(space, 'ObjectPlacement'):
                        windows_in_space.append(window_properties[window])
                        total_window_area += window_properties[window]["area"]
                except:
                    pass
        
        # Calculate window-to-floor ratio
        window_ratio = 0.0
        if space_area and space_area > 0:
            window_ratio = total_window_area / space_area
        
        # Check if the requirement is met (min 12.5%)
        requirement_met = window_ratio >= report["summary"]["min_ratio"]
        
        report["windows_by_space"][space_name] = {
            "total_windows": len(windows_in_space),
            "total_window_area": total_window_area,
            "floor_area": space_area,
            "window_ratio": window_ratio,
            "ratio_percentage": window_ratio * 100 if space_area else 0,
            "windows": windows_in_space
        }
        
        report["compliance_status"][space_name] = {
            "compliant": requirement_met,
            "required_ratio": report["summary"]["min_ratio"],
            "actual_ratio": window_ratio,
            "status": "✓ PASS" if requirement_met else "✗ FAIL"
        }
        
        if requirement_met:
            report["summary"]["compliant_spaces"] += 1
        else:
            report["summary"]["non_compliant_spaces"] += 1
    
    return report


# Test your implementation
analysis = analyze_window_compliance(ifc, spaces)

print("="*70)
print("WINDOW COMPLIANCE ANALYSIS REPORT")
print("="*70)
print(f"\nTotal Windows Found: {analysis['total_windows']}")
print(f"Requirement: Window area ≥ 12.5% of floor area\n")

print("-"*70)
for space_name, compliance in analysis["compliance_status"].items():
    status = compliance["status"]
    ratio = compliance["actual_ratio"] * 100
    required = compliance["required_ratio"] * 100
    print(f"{space_name:<12} | {status} | Ratio: {ratio:5.1f}% (Required: {required:.1f}%)")

print("-"*70)
print(f"\n✓ Compliant: {analysis['summary']['compliant_spaces']} spaces")
print(f"✗ Non-Compliant: {analysis['summary']['non_compliant_spaces']} spaces")


Found 24 windows in the model

WINDOW COMPLIANCE ANALYSIS REPORT

Total Windows Found: 24
Requirement: Window area ≥ 12.5% of floor area

----------------------------------------------------------------------
A101         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
A201         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B104         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B101         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B201         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
A205         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B205         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
A105         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B105         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
R301         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B102         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
A102         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
A103         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B103         | ✗ FAIL | Ratio:   0.0% (Required: 12.5%)
B204   

## Bonus Exercise: Fire Safety Route Analysis

**Objective:** Find the longest evacuation route within the apartment and verify it meets fire safety requirements.

**Difficulty:** Advanced

**Reference:** [Catalan Building Code - Fire Safety Section](https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e)

### Fire Safety Requirements

According to the Catalan building code:
- **Maximum travel distance** to exit: ≤ 25-30 m (depending on building type)
- **Minimum corridor width**: 1.2 m
- **Minimum door width**: 0.8 m (for exits)
- **No dead-end corridors** longer than 10 m

### Your Task

Write a function `analyze_evacuation_routes(ifc_model, spaces)` that:

1. **Builds a spatial graph** of the apartment (rooms as nodes, doors/openings as connections)
2. **Calculates distances** between spaces (using area/perimeter as proxy)
3. **Finds the longest route** from any point to the nearest exit
4. **Validates** the route against fire safety requirements
5. **Identifies bottlenecks** (narrow corridors, small doors)

**Hints:**
- Think of spaces as nodes and doors as edges in a graph
- Use BFS/DFS to find longest paths
- Door dimensions can indicate width constraints
- Consider calculating distances based on space geometry
- This is a simplified model - real analysis would use detailed geometry

### Starter Code

```python
def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Fire safety analysis and recommendations
    """
    
    # Get all doors (potential connections between spaces)
    doors = ifc_model.by_type("IfcDoor")
    
    analysis = {
        "total_spaces": len(spaces),
        "total_doors": len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False
    }
    
    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors")
    
    # TODO: Build spatial connectivity graph
    # TODO: Find longest evacuation path
    # TODO: Check corridor widths and door dimensions
    # TODO: Validate against requirements
    # TODO: Report issues and recommendations
    
    return analysis


# Run the analysis (uncomment when ready)
# fire_analysis = analyze_evacuation_routes(ifc, spaces)
```

In [15]:
# Bonus Exercise: Implement your solution here

from collections import deque, defaultdict
import math

def analyze_evacuation_routes(ifc_model, spaces):
    """
    Analyze evacuation routes and fire safety compliance.
    
    Args:
        ifc_model: The loaded IFC model
        spaces: List of IfcSpace objects
        
    Returns:
        dict: Fire safety analysis and recommendations
    """
    
    # Get all doors (potential connections between spaces)
    doors = ifc_model.by_type("IfcDoor")
    
    analysis = {
        "total_spaces": len(spaces),
        "total_doors": len(doors),
        "longest_route": None,
        "longest_distance": 0,
        "safety_issues": [],
        "compliant": False,
        "door_analysis": {},
        "space_connectivity": {}
    }
    
    print(f"Analyzing {len(spaces)} spaces with {len(doors)} doors\n")
    
    # Build a graph where each space is a node and each door connecting spaces is an edge
    graph = defaultdict(list)  # space_name -> [(connected_space_name, distance, door_width)]
    
    # Extract and check door dimensions
    door_data = {}
    for door in doors:
        door_name = door.Name if door.Name else "Unnamed Door"
        door_width = 0.8  # Default minimum door width
        door_height = 2.0  # Default door height
        
        # Try to extract dimensions from properties
        if hasattr(door, 'Quantities') and door.Quantities:
            for quantity in door.Quantities.Quantities:
                if hasattr(quantity, 'Name'):
                    if quantity.Name == "Width" and hasattr(quantity, 'LengthValue'):
                        door_width = quantity.LengthValue
                    elif quantity.Name == "Height" and hasattr(quantity, 'LengthValue'):
                        door_height = quantity.LengthValue
        
        # Try to extract from geometry if not found
        if door_width == 0.8:
            if hasattr(door, 'Representation') and door.Representation:
                try:
                    coords = []
                    for rep in door.Representation.Representations:
                        if hasattr(rep, 'Items'):
                            for item in rep.Items:
                                if hasattr(item, 'Points'):
                                    for point in item.Points:
                                        if hasattr(point, 'Coordinates'):
                                            coords.append(point.Coordinates)
                    
                    if coords:
                        x_vals = [c[0] for c in coords]
                        y_vals = [c[1] for c in coords]
                        z_vals = [c[2] for c in coords]
                        door_width = max(x_vals) - min(x_vals) if max(x_vals) - min(x_vals) > 0 else 0.8
                        door_height = max(z_vals) - min(z_vals) if max(z_vals) - min(z_vals) > 0 else 2.0
                except:
                    pass
        
        door_data[door_name] = {
            "width": door_width,
            "height": door_height,
            "compliant_width": door_width >= 0.8,  # Min 0.8m for exits
            "object": door
        }
        
        # Check door compliance
        if door_width < 0.8:
            analysis["safety_issues"].append(
                f"Door '{door_name}' width {door_width:.2f}m below minimum 0.8m"
            )
    
    analysis["door_analysis"] = door_data
    
    # Create space lookup
    space_dict = {(space.Name if space.Name else "Unnamed"): space for space in spaces}
    
    # Calculate distance/cost for each connection (simplified)
    # Distance based on space floor area as proxy for room size
    space_areas = {}
    for space_name, space in space_dict.items():
        area = 10.0  # Default area
        if hasattr(space, 'Quantities') and space.Quantities:
            for quantity in space.Quantities.Quantities:
                if hasattr(quantity, 'Name') and quantity.Name == "NetFloorArea":
                    if hasattr(quantity, 'AreaValue'):
                        area = quantity.AreaValue
        space_areas[space_name] = area
    
    # Simple connectivity: assume each door connects spaces adjacently
    # For this simplified model, connect all spaces to build a complete graph
    for i, space1_name in enumerate(space_dict.keys()):
        for j, space2_name in enumerate(space_dict.keys()):
            if i != j:
                # Distance = average of space areas (proxy for room size/travel distance)
                area1 = space_areas[space1_name]
                area2 = space_areas[space2_name]
                # Distance estimation: sqrt(area) gives approximate room dimension
                distance1 = math.sqrt(area1)
                distance2 = math.sqrt(area2)
                avg_distance = (distance1 + distance2) / 2
                
                # Use minimum door width as constraint (assume smallest connected door)
                min_door_width = min([d["width"] for d in door_data.values()]) if door_data else 1.2
                
                graph[space1_name].append((space2_name, avg_distance, min_door_width))
    
    analysis["space_connectivity"] = dict(graph)
    
    # Use BFS to find longest path from any space to nearest exit
    # Assuming first space is near an exit for simplified model
    longest_distance = 0
    longest_route = None
    
    if spaces:
        # For each space, find the longest distance to any other space (simplified evacuation)
        for start_space in space_dict.keys():
            visited = set()
            queue = deque([(start_space, 0, [start_space])])
            visited.add(start_space)
            
            while queue:
                current_space, current_distance, path = queue.popleft()
                
                if current_distance > longest_distance:
                    longest_distance = current_distance
                    longest_route = path
                
                # Explore neighbors
                if current_space in graph:
                    for next_space, distance, door_width in graph[current_space]:
                        if next_space not in visited:
                            visited.add(next_space)
                            queue.append((next_space, current_distance + distance, path + [next_space]))
    
    analysis["longest_distance"] = longest_distance
    analysis["longest_route"] = longest_route
    
    # Check if longest_distance <= 25m (requirement)
    max_travel_distance = 25  # meters
    analysis["compliant"] = longest_distance <= max_travel_distance
    
    # Check corridor widths and door dimensions
    min_corridor_width = 1.2  # meters
    min_exit_width = 0.8  # meters
    
    for door_name, door_info in door_data.items():
        if door_info["width"] < min_exit_width:
            analysis["safety_issues"].append(
                f"Exit door '{door_name}' ({door_info['width']:.2f}m) below minimum {min_exit_width}m"
            )
    
    # Report safety issues and recommendations
    if not analysis["compliant"]:
        analysis["safety_issues"].append(
            f"Longest evacuation route {analysis['longest_distance']:.1f}m exceeds maximum {max_travel_distance}m"
        )
    
    if longest_distance > 20:
        analysis["safety_issues"].append(
            "Consider adding emergency exits to reduce evacuation distance"
        )
    
    if longest_distance > 15:
        analysis["safety_issues"].append(
            "Evacuation route is relatively long - ensure clear pathways"
        )
    
    return analysis


# Test your solution
fire_analysis = analyze_evacuation_routes(ifc, spaces)

print("="*70)
print("FIRE SAFETY EVACUATION ANALYSIS")
print("="*70)
print(f"\nTotal Spaces: {fire_analysis['total_spaces']}")
print(f"Total Doors: {fire_analysis['total_doors']}")
print(f"\nLongest Evacuation Route: {fire_analysis['longest_distance']:.1f}m")
if fire_analysis['longest_route']:
    print(f"Route: {' → '.join(fire_analysis['longest_route'][:5])}...")

print("\n" + "-"*70)
print("DOOR DIMENSIONS ANALYSIS")
print("-"*70)
for door_name, door_info in fire_analysis['door_analysis'].items():
    status = "✓" if door_info['compliant_width'] else "✗"
    print(f"{status} {door_name:<20} | Width: {door_info['width']:.2f}m | Height: {door_info['height']:.2f}m")

print("\n" + "-"*70)
print("COMPLIANCE STATUS")
print("-"*70)
if fire_analysis['compliant']:
    print("✓ COMPLIANT - Evacuation routes meet fire safety requirements")
else:
    print("✗ NON-COMPLIANT - Fire safety issues found:")
    for issue in fire_analysis['safety_issues']:
        print(f"  ⚠ {issue}")

print("\n" + "-"*70)
print("RECOMMENDATIONS")
print("-"*70)
if fire_analysis['longest_distance'] > 25:
    print("• Add intermediate exits to reduce evacuation distance")
if any(not d['compliant_width'] for d in fire_analysis['door_analysis'].values()):
    print("• Widen doors to meet minimum 0.8m exit requirement")
print("• Ensure all corridors are clear of obstacles")
print("• Install emergency lighting along evacuation routes")
print("• Post evacuation route maps in prominent locations")


Analyzing 21 spaces with 14 doors

FIRE SAFETY EVACUATION ANALYSIS

Total Spaces: 21
Total Doors: 14

Longest Evacuation Route: 3.2m
Route: A101 → A201...

----------------------------------------------------------------------
DOOR DIMENSIONS ANALYSIS
----------------------------------------------------------------------
✓ M_Single-Flush:1250mm x 2010mm:1250mm x 2010mm:146596 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:1250mm x 2010mm:1250mm x 2010mm:146678 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0762 x 2032mm:0762 x 2032mm:150173 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0762 x 2032mm:0762 x 2032mm:150257 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0864 x 2032mm:0864 x 2032mm:150378 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0864 x 2032mm:0864 x 2032mm:150478 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0864 x 2032mm:0864 x 2032mm:159734 | Width: 0.80m | Height: 2.00m
✓ M_Single-Flush:0864 x 2032mm:0864 x 2032mm:159834 | Width: 0.80m | Height: 2.00m
✓ M_S

## Useful IFC Concepts

### Common Entity Types

- **IfcSpace**: A room, area, or zone in the building
- **IfcWindow**: Windows for light and ventilation  
- **IfcDoor**: Doors, openings, access points
- **IfcWall**: Boundary elements
- **IfcBuildingElement**: General building components
- **IfcElement**: Physical building pieces with properties

### Accessing Properties

```python
# Get all entities of a type
elements = ifc.by_type("IfcWindow")

# Access properties
entity.Name                    # String name
entity.Description             # Text description
entity.ObjectPlacement        # Location/coordinates
entity.Representation         # Geometry data
entity.QuantityInSpace         # Calculate-derived quantities

# Common relationships
entity.HasProperties           # Get properties
entity.BoundedBy               # Spatial boundaries
entity.HostedBy                # Connection relationships
```

### Helpful Methods

```python
# Query by GUID
entity = ifc.by_id(guid)

# Filter by property value
elements = ifc.by_attribute("Name", "Kitchen")

# Get all instances of a type
spaces = ifc.by_type("IfcSpace")

# Check entity type
if space.is_a("IfcSpace"):
    print("This is a space")
```

## Resources for Help

- **IFC Knowledge Base**: https://notebooklm.google.com/notebook/0925c2a1-519b-40a8-aca4-1e832d219f3c
- **IfcOpenShell Documentation**: https://docs.ifcopenshell.org/ifcopenshell-python.html
- **BuildingSmart Standards**: https://www.buildingsmart.org/
- **Catalan Building Code**: https://notebooklm.google.com/notebook/216b245f-0fc1-4063-bdfd-d23b41360b0e

## Next Steps

After completing these exercises, you'll have learned:
- ✓ How to load and explore IFC files
- ✓ How to extract spatial and building data
- ✓ How to validate designs against building codes
- ✓ How to work with doors, windows, and routes

These skills apply to real-world AEC (Architecture, Engineering, Construction) workflows.